# 字节跳动Doubao嵌入模型

这将帮助您使用LangChain开始使用字节跳动Doubao [嵌入模型](/docs/concepts/embedding_models)。有关`ByteDanceDoubaoEmbeddings`功能和配置选项的详细文档，请参阅[API参考](https://api.js.langchain.com/classes/_langchain_community.embeddings_bytedance_doubao.ByteDanceDoubaoEmbeddings.html)。

## 概述
### 集成详情

| 类 | 包 | 本地 | Python支持 | 包下载量 | 包最新版本 |
| :--- | :--- | :---: | :---: |  :---: | :---: |
| [ByteDanceDoubaoEmbeddings](https://api.js.langchain.com/classes/_langchain_community.embeddings_bytedance_doubao.ByteDanceDoubaoEmbeddings.html) | [@langchain/community](https://api.js.langchain.com/modules/_langchain_community.html) | ❌ | ❌ | ![NPM - 下载量](https://img.shields.io/npm/dm/@langchain/community?style=flat-square&label=%20&) | ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## 准备

您需要注册一个[ARK API密钥](https://console.volcengine.com/ark/region:ark+cn-beijing/apiKey)，并将其设置为名为`ARK_API_KEY`的环境变量。然后，您应该为嵌入模型[创建一个入口点](https://console.volcengine.com/ark/region:ark+cn-beijing/endpoint)，并使用入口点的名称作为`model`。

接着，您需要安装 [`@langchain/community`](https://www.npmjs.com/package/@langchain/community) 包。

### 凭证

如果您希望自动追踪模型调用，也可以取消下面的注释来设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```bash
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="your-api-key"
```

### 安装

LangChain 字节跳动Doubao嵌入模型集成位于 `@langchain/community` 包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community
</Npm2Yarn>
```

## 实例化

现在我们可以实例化我们的模型对象并嵌入文本：

In [1]:
import { ByteDanceDoubaoEmbeddings } from "@langchain/community/embeddings/bytedance_doubao";

const embeddings = new ByteDanceDoubaoEmbeddings({
  model: 'ep-xxx-xxx' // your entrypoint's name
});

## 索引与检索

嵌入模型通常用于检索增强生成（RAG）流程中，既用于数据索引，也用于后续的数据检索。有关更详细的说明，请参阅我们位于[使用外部知识教程](/docs/tutorials/#working-with-external-knowledge)下的RAG教程。

下面展示如何使用我们前面初始化的`embeddings`对象进行数据索引和检索。在此示例中，我们将使用演示用的[`MemoryVectorStore`](/docs/integrations/vectorstores/memory)来索引和检索一个示例文档。

In [2]:
// Create a vector store with a sample text
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const text = "LangChain is the framework for building context-aware reasoning applications";

const vectorstore = await MemoryVectorStore.fromDocuments(
  [{ pageContent: text, metadata: {} }],
  embeddings,
);

// Use the vector store as a retriever that returns a single document
const retriever = vectorstore.asRetriever(1);

// Retrieve the most similar text
const retrievedDocuments = await retriever.invoke("What is LangChain?");

retrievedDocuments[0].pageContent;

LangChain is the framework for building context-aware reasoning applications


## 直接使用

在底层，向量存储和检索器的实现分别通过调用 `embeddings.embedDocument(...)` 和 `embeddings.embedQuery(...)` 来为 `fromDocuments` 和检索器的 `invoke` 操作中使用的文本创建嵌入。

您可以直接调用这些方法，以用于您自己的用例。

### 嵌入单个文本

您可以使用 `embedQuery` 对查询进行嵌入以用于搜索。这将生成特定于该查询的向量表示：

In [3]:
const singleVector = await embeddings.embedQuery(text);

console.log(singleVector.slice(0, 100));

[
   0.026051683,   0.029081265,  -0.040726297,  -0.015116953, -0.010691089,
   0.030181013, -0.0065084146,   -0.02079503,   0.013575795,   0.03452527,
   0.009578291,   0.007026421,  -0.030110886,   0.013489622,  -0.04294787,
   0.011141899,  -0.043768786,   -0.00362867, -0.0081198225,  -0.03426076,
   0.010075142,   0.027787417,   -0.09052663,   -0.06039698, -0.009462592,
    0.06232288,   0.051121354,   0.011977532,   0.089046724,  0.059000008,
   0.031860664,  -0.034242127,   0.020339863,   0.011483523,  -0.05429335,
   -0.04963588,    0.03263794,   -0.05581542,   0.013908403, -0.012356067,
  -0.007802118,  -0.010027855,    0.00281217,  -0.101886116, -0.079341754,
   0.011269771,  0.0035983133,  -0.027667878,   0.032092705, -0.052843474,
  -0.045283325,     0.0382421,     0.0193055,   0.011050924,  0.021132186,
  -0.037696265,  0.0006107435,  0.0043520257,  -0.028798066,  0.049155913,
    0.03590549, -0.0040995986,   0.019772101,  -0.076119535, 0.0031298609,
    0.03368174,   0.039

### 嵌入多个文本

您可以使用 `embedDocuments` 对多个文本进行索引嵌入。此方法的内部实现可能（但不一定）与嵌入查询的方式不同：

In [4]:
const text2 = "LangGraph is a library for building stateful, multi-actor applications with LLMs";

const vectors = await embeddings.embedDocuments([text, text2]);

console.log(vectors[0].slice(0, 100));
console.log(vectors[1].slice(0, 100));

[
   0.026051683,   0.029081265,  -0.040726297,  -0.015116953, -0.010691089,
   0.030181013, -0.0065084146,   -0.02079503,   0.013575795,   0.03452527,
   0.009578291,   0.007026421,  -0.030110886,   0.013489622,  -0.04294787,
   0.011141899,  -0.043768786,   -0.00362867, -0.0081198225,  -0.03426076,
   0.010075142,   0.027787417,   -0.09052663,   -0.06039698, -0.009462592,
    0.06232288,   0.051121354,   0.011977532,   0.089046724,  0.059000008,
   0.031860664,  -0.034242127,   0.020339863,   0.011483523,  -0.05429335,
   -0.04963588,    0.03263794,   -0.05581542,   0.013908403, -0.012356067,
  -0.007802118,  -0.010027855,    0.00281217,  -0.101886116, -0.079341754,
   0.011269771,  0.0035983133,  -0.027667878,   0.032092705, -0.052843474,
  -0.045283325,     0.0382421,     0.0193055,   0.011050924,  0.021132186,
  -0.037696265,  0.0006107435,  0.0043520257,  -0.028798066,  0.049155913,
    0.03590549, -0.0040995986,   0.019772101,  -0.076119535, 0.0031298609,
    0.03368174,   0.039

## 相关内容

- 嵌入模型 [概念指南](/docs/concepts/embedding_models)
- 嵌入模型 [操作指南](/docs/how_to/#embedding-models)

## API 参考文档

有关所有 ByteDanceDoubaoEmbeddings 特性和配置的详细文档，请访问 API 参考：https://api.js.langchain.com/classes/_langchain_community.embeddings_bytedance_doubao.ByteDanceDoubaoEmbeddings.html